## Content Based Recommender on Movie Taglines and Description

In [2]:
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('D:\\Shubhi\\DataScience\\MoviesRecommendation\\MovieLens_Dataset\\movies_metadata.csv')
df.info()
# Since 'id' is a generic object and not numeric(int/float) shows it contains some non-numeric values. 
# Since when we look for not-null values, it returns 0 rows 
df[df['id'].isnull()].shape

C:\Users\Ridaan\AppData\Local\Temp\ipykernel_15024\1526388219.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:\\Shubhi\\DataScience\\MoviesRecommendation\\MovieLens_Dataset\\movies_metadata.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

(0, 24)

In [8]:
# Find out where id has non-numeric values and hence wil drop them 
df['id'] = pd.to_numeric(df['id'], errors = 'coerce')
#df['id'] = df['id'].replace(0, np.nan) # not necessary. The above line already conerted non-numeric value to NaN
df[df['id'].isnull()].shape
df[df['id'].isnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{""name"": ""Carousel Productions"", ""id"": 11176}...","[{""iso_3166_1"": ""CA"", ""name"": ""Canada""}, {""iso...",NaN,0,104,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,...,01-01-1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{""name"": ""Aniplex"", ""id"": 2883}, {""name"": ""Go...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",NaN,0,68,"[{""iso_639_1"": ""ja"", ""name"": ""日本語""}]",Released,...,12-01-1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{""name"": ""Odyssey Media"", ""id"": 17161}, {""nam...","[{""iso_3166_1"": ""CA"", ""name"": ""Canada""}]",NaN,0,82,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,...,22-01-1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df = df.drop([19730, 29503, 35587])
df['id'] = df['id'].astype('int')

In [11]:
links_small = pd.read_csv('D:\\Shubhi\\DataScience\\MoviesRecommendation\\MovieLens_Dataset\\links_small.csv')
links_small.info()
links_small.head(5)
print(f" Total no. of rows in links_small: {links_small.shape}")
print(f" Count of null values in tmdbId column: {links_small[links_small['tmdbId'].isnull()].shape[0]}")

links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

smd = df[df['id'].isin(links_small)]
print(f" Total no. of common movies in small dataset which matches with those in main dataset: {smd.shape}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB
 Total no. of rows in links_small: (9125, 3)
 Count of null values in tmdbId column: 13
 Total no. of common movies in small dataset which matches with those in main dataset: (9099, 24)


In [18]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')


C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\2267313130.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['tagline'] = smd['tagline'].fillna('')
C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\2267313130.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['description'] = smd['overview'] + smd['tagline']
C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\2267313130.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [19]:
# creating a TfidVectorizer for the description field to give the weightage of all the important words all over the documents (movie descriptions)
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])


In [20]:
tfidf_matrix.shape


(9099, 268123)

In [21]:
# calculating the cosine similarity to measure the pairwise similarity between two movies description
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(9099, 9099)

In [22]:
# creating a series of movies original indexes (to retain the indexes) whose indices will be movie name.   
smd = smd.reset_index()
titles = smd[{'title','vote_count', 'vote_average'}]
indices = pd.Series(smd.index, index=smd['title'])


C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\3290043544.py:3: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  titles = smd[{'title','vote_count', 'vote_average'}]


In [23]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx])) # converting a row into enumerated list items ({(0,.89), (1,.43), ...})
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # sorting based on similarilty score i.e x[1]
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores] # array of top 30 similar movies indices
    return titles.iloc[movie_indices]
    #return pd.concat([titles.iloc[movie_indices], smd.iloc[movie_indices]['director']], axis = 1) # retrieving titles of the movies

In [25]:
get_recommendations('The Godfather').head(10)


,title,vote_average,vote_count
973,The Godfather: Part II,8.3,3418.0
8387,The Family,6.1,1052.0
3509,Made,6.3,55.0
4196,Johnny Dangerously,6.3,67.0
29,Shanghai Triad,6.5,17.0
5667,Fury,7.5,38.0
2412,American Movie,7.7,57.0
1582,The Godfather: Part III,7.1,1589.0
4221,8 Women,6.9,197.0
2159,Summer of Sam,6.3,115.0


## Content Based Recommendor based on Movie Cast, crew(director) and keywords

In [26]:
credits = pd.read_csv('D:\\Shubhi\\DataScience\\MoviesRecommendation\\MovieLens_Dataset\\credits.csv')
keywords = pd.read_csv('D:\\Shubhi\\DataScience\\MoviesRecommendation\\MovieLens_Dataset\\keywords.csv')



In [27]:
# merging cast and crew information in a one dataset. No. of rows returned are more due to the fact tha datasets contain duplicate rows
print(df.shape)
df = df.merge(credits, on='id')
df = df.merge(keywords, on='id')
print(df.shape)


(45463, 24)
(46628, 27)


In [28]:
# creating a subset of dataset
smd = df[df['id'].isin(links_small)]
smd.shape

(9219, 27)

In [29]:
from ast import literal_eval

smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['genres'] = smd['genres'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\1756650891.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(literal_eval)
C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\1756650891.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['crew'] = smd['crew'].apply(literal_eval)
C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\1756650891.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

#### Important points while creating metadata
1. From crew, only taking Director and then repeating it 3 times to increase the weightage over casting (heros, etc.) while creating CountVectorizer 
2. Taking only top 3 casting characters
3. Strip Spaces and Convert to Lowercase from all our features. This way, our engine will not confuse between Johnny Depp and Johnny Galecki.

In [30]:
def get_Director(x): 
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [31]:
# Point 1
smd['director'] = smd['crew'].apply(get_Director)

C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\3030668650.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['director'] = smd['crew'].apply(get_Director)


In [32]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else [])

C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\1782956766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else [])


In [33]:
smd['genres'] = smd['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\3514851865.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['genres'] = smd['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


In [34]:
# Point 2
print(smd['cast'][0])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
print(smd['cast'][0])

['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim Varney', 'Wallace Shawn', 'John Ratzenberger', 'Annie Potts', 'John Morris', 'Erik von Detten', 'Laurie Metcalf', 'R. Lee Ermey', 'Sarah Freeman', 'Penn Jillette']
['Tom Hanks', 'Tim Allen', 'Don Rickles']


C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\1691833001.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)


In [35]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else [])

C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\1252475430.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else [])


In [36]:
# Point 3
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
# Point 1
smd['director'] = smd['director'].apply(lambda x: [x,x, x])



C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\3468359161.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\3468359161.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\3468359161.py:5: SettingWithCopyWarning: 
A value is

In [37]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s.head(20)


C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\3378524604.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)


0                    jealousy
0                         toy
0                         boy
0                  friendship
0                     friends
0                     rivalry
0               boy next door
0                     new toy
0           toy comes to life
1                  board game
1               disappearance
1    based on children's book
1                    new home
1                     recluse
1                giant insect
2                     fishing
2                 best friend
2        duringcreditsstinger
2                     old men
3              based on novel
Name: keyword, dtype: object

In [38]:
## counting occurence of every keyword
s = s.value_counts()
print(f' The no. of different keywords: {s.shape}')
s[:5]

 The no. of different keywords: (12940,)


independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [39]:
## calculate how many keywords come only once
s[s==1].shape

(6231,)

In [40]:
## removing those keywords who are occuring only 1 time in the whole dataset of movies
s = s[s > 1]
s.shape

(6709,)

In [41]:
smd.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,title,video,vote_average,vote_count,cast,crew,keywords,cast_size,crew_size,director
0,FALSE,"{""id"": 10194, ""name"": ""Toy Story Collection"", ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy""s toys live happily in his ...",...,Toy Story,False,7.7,5415.0,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...",13,106,"[johnlasseter, johnlasseter, johnlasseter]"
1,FALSE,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Jumanji,False,6.9,2413.0,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...",26,16,"[joejohnston, joejohnston, joejohnston]"
2,FALSE,"{""id"": 119050, ""name"": ""Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Grumpier Old Men,False,6.5,92.0,"[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, best friend, duringcreditsstinger, o...",7,4,"[howarddeutch, howarddeutch, howarddeutch]"
3,FALSE,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Waiting to Exhale,False,6.1,34.0,"[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[based on novel, interracial relationship, sin...",10,10,"[forestwhitaker, forestwhitaker, forestwhitaker]"
4,FALSE,"{""id"": 96871, ""name"": ""Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Father of the Bride Part II,False,5.7,173.0,"[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[baby, midlife crisis, confidence, aging, daug...",12,7,"[charlesshyer, charlesshyer, charlesshyer]"


In [42]:
## creating a function to filter the words row wise according to s (we removed those words who occured only once)
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words    

In [43]:
# Finally, we will convert every word to its stem so that words such as Dogs and Dog are considered the same.

from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')
stemmer.stem('dogs')


'dog'

In [44]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])


C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\1494073444.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['keywords'] = smd['keywords'].apply(filter_keywords)
C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\1494073444.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\1494073444.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [45]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
#smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))


C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\115660057.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']


In [46]:
smd['soup'][0]

['jealousi',
 'toy',
 'boy',
 'friendship',
 'friend',
 'rivalri',
 'boynextdoor',
 'newtoy',
 'toycomestolif',
 'tomhanks',
 'timallen',
 'donrickles',
 'johnlasseter',
 'johnlasseter',
 'johnlasseter',
 'Animation',
 'Comedy',
 'Family']

In [47]:
# joining list elements as a string with a separator ' '
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))


C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\3622758973.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))


In [48]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])


In [49]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [50]:
smd = smd.reset_index()
titles = smd[{'title','vote_count','vote_average'}]
indices = pd.Series(smd.index, index=smd['title'])

In [56]:
# Will resuse our get_recommendations function..This time our cosine_scores are changed
get_recommendations('The Godfather').head(10)


,title,vote_average,vote_count
3616,Tucker: The Man and His Dream,6.5,71.0
994,The Godfather: Part II,8.3,3418.0
1346,The Rainmaker,6.7,239.0
3705,The Cotton Club,6.5,71.0
4518,One from the Heart,5.6,20.0
3300,Gardens of Stone,5.5,25.0
1602,The Godfather: Part III,7.1,1589.0
2998,The Conversation,7.5,377.0
5867,Rumble Fish,6.8,141.0
1992,Peggy Sue Got Married,5.9,138.0


In [59]:
get_recommendations('The Dark Knight').head(20)


,title,vote_average,vote_count
8031,The Dark Knight Rises,7.6,9263.0
6218,Batman Begins,7.5,7511.0
6623,The Prestige,8.0,4510.0
2085,Following,7.2,363.0
7648,Inception,8.1,14075.0
4145,Insomnia,6.8,1181.0
3381,Memento,8.1,4168.0
8613,Interstellar,8.1,11187.0
7659,Batman: Under the Red Hood,7.6,459.0
1134,Batman Returns,6.6,1706.0


In [58]:
get_recommendations('Mean Girls').head(10)


,title,vote_average,vote_count
3319,Head Over Heels,5.3,73.0
4763,Freaky Friday,6.0,919.0
1329,The House of Yes,6.3,30.0
6277,Just Like Heaven,6.5,595.0
7905,"Mr. Popper""s Penguins",5.7,775.0
7332,Ghosts of Girlfriends Past,5.6,716.0
6959,The Spiderwick Chronicles,6.3,593.0
8883,The DUFF,6.8,1372.0
6698,"It""s a Boy Girl Thing",6.3,279.0
7377,"I Love You, Beth Cooper",5.2,204.0


In [81]:
def weighted_rating(x, mi, Co):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+mi) * R) + (mi/(mi+v) * Co)

In [82]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    print(f"Minimum votes required to qualify 60th percentile: {m}")
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, args = (m,C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified


In [83]:
improved_recommendations('The Dark Knight')


Minimum votes required to qualify 60th percentile: 1287.3999999999996


C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\3217831010.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_count'] = qualified['vote_count'].astype('int')
C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\3217831010.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_average'] = qualified['vote_average'].astype('int')
C:\Users\Ridaan\AppData\Local\Temp\ipykernel_20348\3217831010.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

,title,vote_count,vote_average,wr
7648,Inception,14075,8,7.872621
8613,Interstellar,11187,8,7.843131
6623,The Prestige,4510,8,7.662461
3381,Memento,4168,8,7.641301
8031,The Dark Knight Rises,9263,7,6.936548
6218,Batman Begins,7511,7,6.923913
1134,Batman Returns,1706,6,6.206438
132,Batman Forever,1529,5,5.676520
9024,Batman v Superman: Dawn of Justice,7189,5,5.224783
1260,Batman & Robin,1447,4,5.167624
